In [1]:
import pandas as pd
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import os
from sqlalchemy import create_engine
from sklearn.model_selection import KFold

In [2]:
exists=os.path.isfile(os.getcwd())
if exists:
    df_table =pd.read_csv('energy_table.csv').dropna()
else:
    USER = "georgia"
    engine = create_engine(f"postgres+psycopg2://{USER}@energy-link.ct4qqgmkkpxw.eu-west-1.rds.amazonaws.com:5432/energylink")
    df_table = pd.read_sql_table(table_name="energy_table", schema="energylink", con=engine)


In [3]:
#check number of 0 values for the target
(df_table.solar_potential==0).sum()/df_table.solar_potential.shape[0]

0.4944485664568552

Since a big portion of the data is 0s, the best aproach is to guess the times that no energy is produced by the solar panels

Since, except special ocasions, this is mostly due the time of the day, this wil be a clasification model depending on the time of day and date

First we must parse date and time to use on random forest models

In [4]:
df_table['month'] = df_table.local.apply(lambda v: int(v[5:7]))
df_table['day'] = df_table.local.apply(lambda v: int(v[8:10]))
df_table['hour'] = df_table.local.apply(lambda v: int(v[11:13]))



Now is time to create the "target": does it receives any energy? (0: no, 1:yes)

In [5]:
df_table['gets_energy'] = df_table.solar_potential.apply(lambda v:1 if v>0 else 0)

## Clasification model

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np

In [7]:
X1 = df_table[['month','day','hour']].copy()
y1 = df_table['gets_energy'].copy()


In [8]:

kf = KFold(n_splits=10,random_state=314)
classifier_model = {}
split_n = 0
y_pred_class = pd.Series()
for train_index, test_index in kf.split(X1):
    X_train = X1.loc[train_index,:].dropna()
    y_train = y1.loc[train_index].dropna()
    X_test = X1.loc[test_index,:].dropna()
    y_test = y1.loc[test_index].dropna()
    
    classifier = RandomForestClassifier()
    classifier.fit(X_train,y_train)
    classifier_model[split_n]={'model': classifier,
                               'X_train': X_train,
                               'X_test': X_test,
                               'y_train': y_train,
                               'y_test': y_test
                              }
    
    y_pred_class = y_pred_class.append(pd.Series(classifier.predict(X_test),index=y_test.index))
    split_n=split_n+1

In [9]:
(y_pred_class-y1).sum() / y1.shape[0]

0.00850884566995774

In [10]:
y1.shape[0]-y_pred_class.shape[0] #error to fix if applicable

0

# Regresion Model

In [11]:
X2 = df_table[['irradiation','temperature','cloud','month','day','hour']].copy()
y2 = df_table['solar_potential'].copy()  
X2 = X2[y2!=0]
y2 = y2[y2!=0]

split_n = 0
y_pred_regr = pd.Series()

regresion_model = {}
for train_index, test_index in kf.split(X2):

    X_train = X2.loc[train_index,:].dropna()
    y_train = y2.loc[train_index].dropna()
    X_test = X2.loc[test_index,:].dropna()
    y_test = y2.loc[test_index].dropna()

    regr = RandomForestRegressor()
    regr.fit(X_train,y_train)
    regresion_model[split_n]={'model': regr,
                              'X_train': X_train,
                              'X_test': X_test,
                              'y_train': y_train,
                              'y_test': y_test
                              }
    
    y_pred_regr = y_pred_regr.append(pd.Series(regr.predict(X_test),index=y_test.index))
    split_n=split_n+1

Now lets train a simple regresion random forest model

In [12]:
df_table['clas_pred'] = y_pred_class 
df_table['reg_pred'] = y_pred_regr 
df_table = df_table.fillna(0)
df_table['prediction'] = df_table['clas_pred'] *df_table['reg_pred']


# Daily error analysis

In [13]:
result_grouped = df_table[['prediction','solar_potential','month','day']].groupby(['month','day']).sum()


In [14]:
((result_grouped['prediction'] / result_grouped['solar_potential'] -1)*100).describe()

count    366.000000
mean     -28.275081
std       63.249995
min     -100.000000
25%      -63.061935
50%      -47.171082
75%      -19.934131
max      372.304683
dtype: float64

## forecasted data


In [35]:

df_table_forecasted = pd.read_sql_table(table_name="future", schema="energylink", con=engine)
#df_table_input =pd.read_csv('input_data.csv').dropna()

In [36]:
df_table_forecasted['month'] = df_table_forecasted.local.apply(lambda v: int(v[5:7]))
df_table_forecasted['day'] = df_table_forecasted.local.apply(lambda v: int(v[8:10]))
df_table_forecasted['hour'] = df_table_forecasted.local.apply(lambda v: int(v[11:13]))

df_table_forecasted.drop(['time', 'id', 'local','solar_potential'], axis=1, inplace=True)
